## Lab 13: Performance Metrics for Classifiers
In Data 8 students learn about performance metrics for continuous outcomes; the most prominent of these was Root Mean Square Error (RMSE), or how far away on average the model's prediction is from the actual value of the outcome. For classifers it is a bit more complicated, but we still want to know how well our model performed. 

Here we will use one year of Nashville traffic stop data--2013, sort of the "high tide" of traffic stops on the part of the Nashville police, who recorded almost 400k stops that year. We will return to what we were working on in Lab 11 and try to predict searches and measure how well our classifer does. Then we will add geospatial data (in the form of census block group, since we do not have a map of organic Nashville neighborhoods) to see if we can get a better prediction of whether or not a stop resulted in a search. Finally, we will take advantage of the geospatial data for a map visualization of searches.

In [ ]:
# dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
!pip install scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, LinearRegression
from sklearn.model_selection import KFold
# from sklearn.decomposition import PCA
import sklearn.linear_model as linear
import sklearn.svm as svm
import sklearn.tree as tree
import sklearn.ensemble as ensemble
import sklearn.neighbors as neighbors
import sklearn.metrics as metrics

import seaborn as sns
!pip install geopandas
!pip install rtree
import geopandas as gpd
import fiona
!pip install plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff
from IPython.display import display, HTML # makes the output in Jupyter notebook pretty
import plotly.express as px
import json
import os

# set the random seed so that results reproduce
np.random.seed(10)

## 1. Prepare the data
For this lab we will use the file of Nashville stop data, rather than the sample used in the previous two labs. We will do what we did before to ready the data for fitting the model.
* read the datafile for 2013 into a dataframe
* clean the data and deal with any missing data that will affect the analysis
* select the features you used in Lab 11 and make them into a dataframe
* split the data into training, validation, and test sets
* scale the data

The 2013 data will occupy more memory than the sample but we still should be fine.

In [ ]:
# load the cleaned data for 2013 only

path = "https://github.com/jdmarshl/data/raw/main/2013_stops_cleaned.csv"
stops_2013 = pd.read_csv(path, index_col=0)


In [ ]:
stops_2013.info()

In [ ]:
# find proportions of missing data

pd.options.display.float_format = '{:.8f}'.format # use 8 decimal places and not scientific notation for float
stops_2013...

In [ ]:
# is there any problematic age data?
stops_2013[stops_2013['subject_age'] > 90]['subject_age'].value_counts()

In [ ]:
# only use rows where the subject_age is less than 99

stops_2013 = ...

**Question**

Other than age, is there any feature we need to be concerned with at this point? Why is it important to get rid of the anomalous `'subject_age'` values?

_your answer here_



In [ ]:
stops_2013.head()

In [ ]:
# select the features that you arrived at in Lab 11 (you can adjust those if you want) 
# to use for predicting search but be sure to include at least the ones listed below
# since we want to see how the model predictions are affected by subject race

# note dummy variables have a reference category, and there is a collinearity problem 
# if *all* the dummy variables are used as predictors at once
reasonable_features = ...
reasonable_df = stops_2013[reasonable_features]
reasonable_matrix = reasonable_df.corr()
plt.figure(figsize=(len(reasonable_features),len(reasonable_features)))
g = sns.heatmap(reasonable_matrix, annot=True, annot_kws={"size": 12})
reasonable_matrix

In [ ]:
# split the data

y = ...
X = ...

# split the sampled data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.80, test_size=0.20)
# split the sampled training set into training and validation sets
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, train_size=0.75, test_size=0.25)

# reminder of size of training, validation, and test sets
print("X_train shape: ", X_train.shape)
print("X_validate shape: ", X_validate.shape)
print("X_test shape: ", X_test.shape)

In [ ]:
# scale the data

scaler = StandardScaler()
scaler.fit(X_train)

# transform all the sets based on the the scaler fit to the training data and make them into df's
# note: you can fit and transform the training set in one step if you want

X_train = ...
X_validate = ...
X_test = ...

#### Baseline or "naive" prediction

If someone were to ask you "Did this randomly drawn traffic stop result in a search," what should your answer be? What is the accuracy of the baseline prediction of "no search"?

In [ ]:
# report the baseline accuracy on the validation to compare results to later
print("Baseline accuracy (proportion search_conducted = False) on validation set is ", ...)

## 2. Construct the logistic regression model

We will again use sklearn's logistic classifier. It will be very hard to beat a naive prediction of "no search" because searches are so uncommon. Because we will want to penalize weak predictors once we add a large number of geospatial features (census block groups), this time we will apply $l2$ regularization to the model. This means that rather than minimizing just the sum of the squared residuals (RSS), we are minimizing the sum of the squared residuals plus a shrinkage penalty. In sklearn's [logistic classifier](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), and in Ridge regression for making estimates of continuous outcome, the penalty is the $l2$ norm. Here the shrinkage penalty is the product of a tuning parameter $\lambda$ and the sum of the squared coefficients. This has the effect of shrinking small coefficients toward zero. Here is the objective function to minimize in math notation.


$$
\sum_{i=1}^{n} \left( y_{i} - \beta_{0} - \sum_{j=1}^{p} \beta_{j}x_{ij} \right) ^2 + \lambda \sum_{j=1}^{p} \beta_{j}^{2} = RSS + \lambda \sum_{j=1}^{p}\beta_j^2
$$


Also, we will use `'class_weight' =  'balanced'` to adjust for the fact that searches are infrequent. We should still keep our enthusiasm in check, because it does not seem as though any of our predictors will help.

In [ ]:
# construct a logistic classifier model using sklearn

logit_cf = ...

# fit the logistic regression model to the training set
logit_cf.fit(X_train, y_train)

# check the accuracy on the training and validation sets
print("Accuracy on training set: ", ...)
print("Accuracy on validation set: ", ...)


## 3. Gauge the performance of the model
All alone, accuracy does not tell us everything we need to know about the performance of a classifier.

For a well-developed explanation (using the language of linear algebra) see the [Data 100 lecture on logistic classifer thresholds and performance measures](https://ds100.org/fa23/resources/assets/lectures/lec23/lec23.html)

This model was far less accurate than the naive prediction. Typically, though, we want to know not just what proportion the model got right, but how good the model is at classifying both $Y = 1$ and $Y = 0$ classes. That is, we want to know True Positives (TP), False Positives (FP), True Negatives (TN), and False Negatives (FN). This can be visualized in a confusion matrix and summarized by the following performance measures
* Precision = $\frac{TP}{TP + FP}$ increases as the number of false positives decreases, as the classification threshold is raised. One way to think of Precision is as what proportion of the class members the classifier identifies are actually members of that class.

* Recall = $\frac{TP}{TP +FN}$ increases as the number of false negatives decreases, as the classification threshold is lowered. One way to think of Recall is as what proportion of the total number of class members does the classifier identify.

* AUC-ROC is the Area Under the Curve - Receiver Operating Characteristic. This is a summary measure of how good a job the classifier does distinguishing between positive and negative class membership at all threshold settings. For example, if we choose one instance at random from the positive class and one at random from the negative class, an AUC-ROC score of 0.6 means that there is a 0.6 probability that the positive instance got a higher score from the classifier.

We will also take advantage of the fact that the logistic classifier returns a probability, and then compares that to a threshold value (for sklearn linear logistic classifier) to predict class membership. Below we will examine the AUC-ROC score, whether the predicted probability of membership in the positive class differs by race, and also what the coefficient of each feature is. 

For more on model performance metrics see the [sklearn documentation](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics).

Note that `.predict_proba` returns an array in which the first col is the probability of False and the second column is the probability of True.

In [ ]:
# helper function to compute statistics across predictors (credit: Vyoma Raman)
def demographics(df, col):
    total = df[col].mean()
    black = df[df.subject_race_black > 0][col].mean()
    nonblack = df[df.subject_race_black <= 0][col].mean()
    male = df[df.subject_sex_male > 0][col].mean()
    nonmale = df[df.subject_sex_male <= 0][col].mean()
    moving_violation = df[df['violation_moving traffic violation'] > 0][col].mean()
    equipment_violation = df[df['violation_vehicle equipment violation'] > 0][col].mean()
    investigative_stop = df[df['violation_investigative stop'] > 0][col].mean()
    print(col, 'average for all stops:', round(total, 5))
    print(col, 'average for black drivers:', round(black, 5))
    print(col, 'average for non-black drivers:', round(nonblack, 5))
    print(col, 'average for male drivers:', round(male, 5))
    print(col, 'average for non-male drivers:', round(nonmale, 5))
    print(col, 'average for moving violations:', round(moving_violation, 5))
    print(col, 'average for equipment violations:', round(equipment_violation, 5))
    print(col, 'average for investigative stops:', round(investigative_stop, 5))

In [ ]:
# get model prediction and the probability estimate for search for each observation in validation set
predictions = ...
probabilities = ...

In [ ]:
# make a new df that has the predictors, model predictions, and probabilities for the validation set
compare = X_validate.copy()
compare['prediction'] = list(predictions) # need to cast as a list because the array above has wrong shape
compare['probability'] = probabilities[:, 1] # the element of the array for probability of true
compare.head()

In [ ]:
# examine the average predicted probability of a search by race and type of stop
demographics(...)

In [ ]:
# what proportion of the demographic groups does the model predict will be searched?
demographics(...)

In [ ]:
# let's illustrate the prediction accuracy with a confusion matrix for the validation set

logit_cf_matrix = metrics.confusion_matrix(...)
logit_cf_matrix

In [ ]:
# display the confusion matrix as a heatmap

#SOLUTION
logit_cf_cm = pd.DataFrame(logit_cf_matrix, range(2), range(2))
logit_cf_cm = logit_cf_cm.rename(index=str, columns={0:'False', 1:'True'})
logit_cf_cm.index = ['False', 'True']
plt.figure(figsize = (10,7))
sns.set(font_scale=1.4)#for label size
sns.heatmap(logit_cf_cm, 
           annot=True,
           fmt = '9.0f',
           annot_kws={"size": 16})

plt.title("Logistic Classifier Confusion Matrix for Search Conducted")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

In [ ]:
# find the logistic regression classifier's recall and precision and AUC-ROC score on the validation set

logit_cf_recall = ...
logit_cf_precision = ...
logit_cf_auc_roc_score = ...
print("logistic regression classifer recall: ",logit_cf_recall)
print("logistic regression classifer precision: ", logit_cf_precision)
print("logistic regression classifer AUC-ROC score: ", logit_cf_auc_roc_score)

In [ ]:
# show which features are most predictive of search (remember that the features have been standardized 
# and it is a logistic regression so you cannot interpret the coefficients as the actual increase in probability)
# you can just zip the features and the .coef_ attribute of the model

feature_coefs = zip(...)
list(feature_coefs)

**Questions:**

1. How good was the logistic regression classifier at predicting whether or not there was a search after a traffic stop? Explain with reference to the metrics above.
2. What could you conclude about the logit classifier if you just looked at the AUC-ROC score? 
3. Write down two interesting pieces of information about predicting search from the exercise above. 

_your answers here_



## 4. Add geospatial data from American Community Survey and add features
So far we have had a tough time predicting searches, but the Nashville traffic stop data has a limited list of features. The literature on policing says that police behavior varies by location, and that police search more in 'high crime' areas. There is also a body of scholarship that says police are more willing to question people when they seem out of place--the "race out of place" hypothesis. It seems reasonable to see if adding some geospatial and demographic information might help us.

Because San Francisco robbed us of a chance to do a geospatial join using geopandas, we will do that here to put each traffic stop into a census block group. That will allow us to see some basic information about the block group like median income and racial composition. I used American Community Survey data from 2013 to add the median income data and data on race. Since Nashville uses different categories for race, I did my best to approximate them. The ACS codes race by asking participants about race and Hispanic ethnicity. In the file that we will use below I coded race as a police officer would ascribe race to someone--regardless of Hispanic ethnicity, white, Black, and API were coded in those catgories, while ACS's category of Hispanic and two or more races was coded as Hispanic. I am not certain but I don't think Nashville police in 2013 asked drivers whom they stopped what their race was--they just filled in a race on a reporting form. So I am assuming it is the police officer making the report who is constructing the "race" category in Nashville.

According to the Census Bureau, the coordinate reference system for the Census Block Groups is GRS_1980, which for GeoPandas is EPSG:7019, but the GeoPandas [reference on coordinate reference systems](https://geopandas.org/en/stable/docs/user_guide/projections.html) says that most of the time you don't need to since data from reputable sources include projection information. It also does not recognize that CRS code. The debugger said that the census block group coordinate reference system is 'EPSG:4269'. We will stick with that  for now. We will have to use [GeoSeries](https://geopandas.org/en/stable/docs/reference/geoseries.html) to make a geometry column for the dataframe with the traffic stop data in it.

There is a risk that we will bump up against memory quotas, or that GeoPandas may not work as expected, but I am keeping my fingers crossed. **Finally, we should be sure that all of the features are of the right type to pass to sklearn. More below.**

In [ ]:
# we have latitudes and longitudes but any mapping is going to want a list of coord pair lists so keep that

lats = ...
longs = ...
coords = ...
reasonable_df['coords'] = coords
reasonable_df.head()

In [ ]:
# first we need to create a GeoSeries geometry object to add the points to our dataframe
# as a shapely.Point geometry, although with .from_xy method
# Note: GeoSeries expects x to be longitude and y to be latitude!

from shapely.geometry import Point
x = ...
y = ...
geopoints = gpd.GeoSeries.from_xy(x, y, crs='EPSG:4269')
len(geopoints)

In [ ]:
# turn the pandas dataframe of selected features into a GeoDataFrame by adding the geometry
# column we made with GeoSeries

reasonable_gdf = gpd.GeoDataFrame(...)
reasonable_gdf.info()

In [ ]:
# read the file of Nashville block groups and associated demographic features into GeoDataFrame
path = 'https://github.com/jdmarshl/data/raw/main/nashville_block_groups.shp'
block_groups = gpd.read_file(path)
block_groups.info()

<span style="color:red">**Looks like some columns need to be cast as different types!**</span>

The ACS data are stored as strings, and so we have to do something to check for missing data and turn the strings into numbers. In particular, we know we want the 2013 median household income `'med_inc_20'` to be an integer and the proportions of populations in the various Nashville racial groups `'white_pct', 'black_pct',v'hispanic_p', 'asianpac_1', 'other_pct'` to be floating point numbers.

In [ ]:
# since the median incomes are strings, we should remove any rows that are spurious
block_groups = block_groups.where(block_groups.med_inc_20.str.len() > 3)
block_groups.info()

In [ ]:
# drop the rows that now have NaNs for 'med_inc_20' and cast the columns as the appropriate data type
# using the pandas method .to_numeric

block_groups.dropna(inplace=True)
block_groups['med_inc_20'] = ...
...
block_groups.info()

In [ ]:
just_blk_groups = block_groups.copy()

In [ ]:
fig = px.histogram(just_blk_groups, x='med_inc_20')
fig.show()

### Joining the traffic stop data and the geospatial data
We now need to connect the traffic stop data, which are represented as points in space. To do this we will use the [GeoPandas spatial join method](https://geopandas.org/en/stable/docs/user_guide/mergingdata.html). Note that there are a number of ways to [make a spatial join](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.sjoin.html#geopandas.GeoDataFrame.sjoin).

The GeoPandas spatial join will allow us to place the point geometry of each traffic stop in the proper polygon that represents the census block group. We can then tie the traffic stop information to the demographics of the block in which it occurred. 

In [ ]:
# geospatial join of the two dataframes defined by polygons and points
block_groups = block_groups.sjoin(...)
block_groups.info()

**Question**

How many rows were in `'reasonable_df'`? How many rows are in the merged GeoDataFrame `'block_groups'`? How do you account for the discrepancy?

_your answer here_


In [ ]:
# a scatterplot of latitudes and longitude can help answer the question above

fig = px.scatter(...)
fig.show()

## 5. Gauge performance of model with added features
Now (with a fair amount of effort) we have added some environmental features, like median household income, of the geographic area of the traffic stop to the individual-level features we had, like `'subject_race'`. We can see if adding the environmental features of a traffic stop improves our ability to predict whether or not the stop led to a search.

This time you may just use the features you used before, which are now in the GeoDataFrame, and add at least the median household income and proportion Black residents as census block group level features. After that, split the data, scale the data, and fit the logistic classifier to the new training set. We can then see whether the performance metrics have improved or not.

In [ ]:
block_groups.columns

In [ ]:
# select features as above but add the environmental features from the census block groups

reasonable_features = [...]
reasonable_df = ...

In [ ]:
# split the data

y = ...
X = ...

# split the sampled data into training and test sets

# split the sampled training set into training and validation sets


# reminder of size of training, validation, and test sets
print("X_train shape: ", X_train.shape)
print("X_validate shape: ", X_validate.shape)
print("X_test shape: ", X_test.shape)

In [ ]:
# scale the data on the training set and apply the scaling to validation and test sets



In [ ]:
# fit the logistic regression model to the training set
# then report the accuracy on the training and validation sets




In [ ]:
# get model prediction and the probability estimate for search for each observation in validation set




In [ ]:
# make a new df that has the predictors, model predictions, and probabilities for the validation set

compare = X_validate.copy()
compare['prediction'] = ...
compare['probability'] = ...
compare.head()

In [ ]:
# report the prediction accuracy with a confusion matrix for the validation set



In [ ]:
# display the confusion matrix as a heatmap



In [ ]:
# report the logistic regression classifier's recall and precision and AUC-ROC score on the validation set



In [ ]:
# show which features are most predictive of search (remember that the features have been standardized so you
# cannot interpret the coefficients as the actual increase in probability)

feature_coefs = 
list(feature_coefs)

**Questions**

1. Did the model perform any better with the environmental features included? 
2. Can you think of other ways to improve the performance of the model?
3. What did this (extended!) exercise teach us about predicting police searches of cars in traffic stops?

_your answers here_

